In [19]:
import os
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

# Load the CSV file into a DataFrame
df = pd.read_csv('SquadPlayerNames_IndianT20League - SquadData_AllTeams.csv')  # Replace with your actual CSV file name

# Define the range for random values
min_value = 10  # Replace with your desired minimum value
max_value = 100  # Replace with your desired maximum value

# Add a new column with random values within the specified range
df['Fantasy_Score'] = np.random.randint(min_value, max_value + 1, size=len(df))
df['probability'] = np.random.rand(len(df))

# Optionally, save the modified DataFrame back to a new CSV file
df.to_csv('SquadPlayerNames_IndianT20League - SquadData_AllTeams.csv', index=False)  # Replace with your desired output file name

# Print the first few rows of the modified DataFrame to confirm the new column
print(df)

    Credits Player Type           Player Name Team  Fantasy_Score  probability
0       8.0        BOWL          Harshit Rana  KKR             54     0.068176
1       6.5         BAT  Angkrish Raghuvanshi  KKR             69     0.012446
2       6.5        BOWL         Vaibhav Arora  KKR             45     0.266233
3       5.5          WK       Luvnith Sisodia  KKR             64     0.701214
4       6.0        BOWL       Mayank Markande  KKR             67     0.692118
5       6.0        BOWL       Chetan Sakariya  KKR             21     0.471903
6       7.5          WK    Rahmanullah Gurbaz  KKR             57     0.764823
7       6.5        BOWL       Spencer Johnson  KKR             18     0.506644
8       9.0        BOWL    Varun Chakravarthy  KKR             77     0.189489
9       8.0        BOWL         Anrich Nortje  KKR             15     0.600582
10      6.0         ALL   Anukul Sudhakar Roy  KKR             33     0.116088
11      7.5         ALL       Ramandeep Singh  KKR  

In [ ]:
import pulp
import pandas as pd

def select_dream_team(players_df):

    # Create a copy of the DataFrame to avoid modifying the original
    players_df = players_df.copy()

    # Add 10 points to Fielders
    fielding_players = {
        ("Virat Kohli", "RCB"),
        ("Ajinkya Rahane", "KKR"), 
        ("Rovman Powell", "KKR"), 
        ("Philip Salt", "RCB"),  # Remove if he is the WK
        ("Liam Livingstone", "RCB"),  
        ("Tim David", "RCB"),   
        ("Andre Russell", "KKR"), 
        ("Rinku Singh", "KKR"), 
        ("Venkatesh Iyer", "KKR"),
        ("Quinton de Kock", "KKR")
    }

    # Add 10 points to WK
    players_df.loc[players_df["Player Type"] == "WK", "Fantasy_Score"] += 10
    
    # Boost Fantasy Score for specific players
    players_df.loc[
        players_df[["Player Name", "Team"]].apply(tuple, axis=1).isin(fielding_players), 
        "Fantasy_Score"
    ] += 10

    players = players_df.to_dict('records')
    prob = pulp.LpProblem("Dream_11_IPL_Team_Selection", pulp.LpMaximize)

    num_players = len(players)

    # Decision variables:
    x = pulp.LpVariable.dicts("Select", range(num_players), cat=pulp.LpBinary)
    c = pulp.LpVariable.dicts("Captain", range(num_players), cat=pulp.LpBinary)
    v = pulp.LpVariable.dicts("ViceCaptain", range(num_players), cat=pulp.LpBinary)

    # Constraints
    prob += pulp.lpSum([x[i] for i in range(num_players)]) == 11, "Total_players"
    prob += pulp.lpSum([players[i]["Credits"] * x[i] for i in range(num_players)]) <= 100, "Credit_limit"
    prob += pulp.lpSum([c[i] for i in range(num_players)]) == 1, "One_captain"
    prob += pulp.lpSum([v[i] for i in range(num_players)]) == 1, "One_vice_captain"

    for i in range(num_players):
        prob += c[i] <= x[i], f"Captain_if_selected_{i}"
        prob += v[i] <= x[i], f"ViceCaptain_if_selected_{i}"
        prob += c[i] + v[i] <= x[i], f"No_double_Designation_{i}"

    teams = set(player["Team"] for player in players)
        
    # At least 1 player from each Player Type category (except BOWL)
    for role in ["WK", "BAT", "ALL", "BOWL"]:
        prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Player Type"] == role]) >= 1, f"Role_{role}_constraint"

    # # Exactly 3 bowlers must be selected
    # prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Player Type"] == "BOWL"]) == 3, "Role_BOWL_constraint"

    # At least 1 bowler from each team
    for team in teams:
        prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Team"] == team and players[i]["Player Type"] == "BOWL"]) >= 1, f"At_least_one_bowler_from_{team}"


    
    # # Objective function: maximize points without considering probability
    objective = pulp.lpSum([
        players[i]["Fantasy_Score"] * x[i] + players[i]["Fantasy_Score"] * c[i] + 0.5 * players[i]["Fantasy_Score"] * v[i]
        for i in range(num_players)
    ])
    # Objective function: maximize points considering both Fantasy Score and Probability
    # objective = pulp.lpSum([
    #     players[i]["Fantasy_Score"] * players[i]["probability"] * x[i] +  # Weighted by probability
    #     players[i]["Fantasy_Score"] * c[i] +  # Captain bonus
    #     0.5 * players[i]["Fantasy_Score"] * v[i]  # Vice-captain bonus
    #     for i in range(num_players)
    # ])

    prob += objective

    # solver = pulp.PULP_CBC_CMD(msg=0)
    solver = pulp.GLPK_CMD(msg=0)
    prob.solve(solver)

    if pulp.LpStatus[prob.status] == "Optimal":
        selected_players = []
        for i in range(num_players):
            if x[i].varValue == 1:
                designation = ""
                if c[i].varValue == 1:
                    designation = "(C)"
                elif v[i].varValue == 1:
                    designation = "(VC)"
                
                selected_players.append({
                    "Player Name": players[i]["Player Name"],
                    "Team": players[i]["Team"],
                    "Type": players[i]["Player Type"],
                    "Designation": designation
                })
        
        return pd.DataFrame(selected_players)
    else:
        return pd.DataFrame()

team = select_dream_team(df)
print(team)

In [ ]:
import pulp
import pandas as pd

def select_dream_team(players_df):
    # Create a copy of the DataFrame to avoid modifying the original
    players_df = players_df.copy()

    # Add 10 points to Fielders
    fielding_players = {
        ("Virat Kohli", "RCB"),
        ("Ajinkya Rahane", "KKR"), 
        ("Rovman Powell", "KKR"), 
        ("Philip Salt", "RCB"),  # Remove if he is the WK
        ("Liam Livingstone", "RCB"),  
        ("Tim David", "RCB"),   
        ("Andre Russell", "KKR"), 
        ("Rinku Singh", "KKR"), 
        ("Venkatesh Iyer", "KKR"),
        ("Quinton de Kock", "KKR")
    }

    # Add 10 points to WK
    players_df.loc[players_df["Player Type"] == "WK", "Fantasy_Score"] += 10
    
    # Boost Fantasy Score for specific players
    players_df.loc[
        players_df[["Player Name", "Team"]].apply(tuple, axis=1).isin(fielding_players), 
        "Fantasy_Score"
    ] += 10

    players = players_df.to_dict('records')
    prob = pulp.LpProblem("Dream_11_IPL_Team_Selection", pulp.LpMaximize)

    num_players = len(players)

    # Decision variables:
    x = pulp.LpVariable.dicts("Select", range(num_players), cat=pulp.LpBinary)

    # Constraints
    prob += pulp.lpSum([x[i] for i in range(num_players)]) == 11, "Total_players"
    prob += pulp.lpSum([players[i]["Credits"] * x[i] for i in range(num_players)]) <= 100, "Credit_limit"

    # At least 1 player from each Player Type category (except BOWL)
    for role in ["WK", "BAT", "ALL", "BOWL"]:
        prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Player Type"] == role]) >= 1, f"Role_{role}_constraint"

    # At least 1 bowler from each team
    teams = set(player["Team"] for player in players)
    for team in teams:
        prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Team"] == team and players[i]["Player Type"] == "BOWL"]) >= 1, f"At_least_one_bowler_from_{team}"

    # Objective function: maximize points considering both Fantasy Score and Probability
    objective = pulp.lpSum([
        players[i]["Fantasy_Score"] * players[i]["probability"] * x[i]  # Weighted by probability
        for i in range(num_players)
    ])
    prob += objective

    # Solve the problem
    solver = pulp.GLPK_CMD(msg=0)
    prob.solve(solver)

    if pulp.LpStatus[prob.status] == "Optimal":
        selected_players = []
        for i in range(num_players):
            if x[i].varValue == 1:
                selected_players.append({
                    "Player Name": players[i]["Player Name"],
                    "Team": players[i]["Team"],
                    "Type": players[i]["Player Type"],
                    "Score": players[i]["Fantasy_Score"],
                    "Probability": players[i]["probability"]
                })

        # Sort players by score * probability to determine Captain and Vice-Captain
        selected_players.sort(key=lambda x: x["Score"] * x["Probability"], reverse=True)

        # Assign Captain and Vice-Captain
        if len(selected_players) > 0:
            selected_players[0]["Designation"] = "(C)"  # Captain
        if len(selected_players) > 1:
            selected_players[1]["Designation"] = "(VC)"  # Vice-Captain

        return pd.DataFrame(selected_players)
    else:
        return pd.DataFrame()



# Example usage
team = select_dream_team(df)
print(team)

In [21]:
# Pitch Condition Taken into account field

import pulp
import pandas as pd
from io import StringIO


def select_dream_team(players_df, match_id):

    # Writing Necessary Data
    data = [
        (1473438, "Kolkata", 0.456),
        (1473439, "Hyderabad", 0.564),
        (1473440, "Chennai", 0.355),
        (1473441, "Visakhapatnam", 0.364),
        (1473442, "Ahmedabad", 0.741),
        (1473443, "Guwahati", 0.572),
        (1473444, "Hyderabad", 0.564),
        (1473445, "Chennai", 0.355),
        (1473446, "Ahmedabad", 0.741),
        (1473447, "Visakhapatnam", 0.364),
        (1473448, "Guwahati", 0.572),
        (1473449, "Mumbai", 0.904),
        (1473450, "Lucknow", 0.172),
        (1473451, "Bengaluru", 0.588),
        (1473452, "Kolkata", 0.456),
        (1473453, "Lucknow", 0.172),
        (1473454, "Chennai", 0.355),
        (1473455, "Mohali", 0.593),
        (1473456, "Kolkata", 0.456),
        (1473457, "Hyderabad", 0.564),
        (1473458, "Mumbai", 0.904),
        (1473459, "Mohali", 0.593),
        (1473460, "Ahmedabad", 0.741),
        (1473461, "Bengaluru", 0.588),
        (1473462, "Chennai", 0.355),
        (1473463, "Lucknow", 0.172),
        (1473464, "Hyderabad", 0.564),
        (1473465, "Jaipur", 0.722),
        (1473466, "Delhi", 0.325),
        (1473467, "Lucknow", 0.172),
        (1473468, "Mohali", 0.593),
        (1473469, "Delhi", 0.325),
        (1473470, "Mumbai", 0.904),
        (1473471, "Bengaluru", 0.588),
        (1473472, "Ahmedabad", 0.741),
        (1473473, "Jaipur", 0.722),
        (1473474, "Mohali", 0.593),
        (1473475, "Mumbai", 0.904),
        (1473476, "Kolkata", 0.456),
        (1473477, "Lucknow", 0.172),
        (1473478, "Hyderabad", 0.564),
        (1473479, "Bengaluru", 0.588),
        (1473480, "Chennai", 0.355),
        (1473481, "Kolkata", 0.456),
        (1473482, "Mumbai", 0.904),
        (1473483, "Delhi", 0.325),
        (1473484, "Jaipur", 0.722),
        (1473485, "Delhi", 0.325),
        (1473486, "Chennai", 0.355),
        (1473487, "Jaipur", 0.722),
        (1473488, "Ahmedabad", 0.741),
        (1473489, "Bengaluru", 0.588),
        (1473490, "Kolkata", 0.456),
        (1473491, "Dharamsala", 0.387),
        (1473492, "Hyderabad", 0.564),
        (1473493, "Mumbai", 0.904),
        (1473494, "Kolkata", 0.456),
        (1473495, "Dharamsala", 0.387),
        (1473496, "Lucknow", 0.172),
        (1473497, "Hyderabad", 0.564),
        (1473498, "Dharamsala", 0.387),
        (1473499, "Delhi", 0.325),
        (1473500, "Chennai", 0.355),
        (1473501, "Bengaluru", 0.588),
        (1473502, "Ahmedabad", 0.741),
        (1473503, "Mumbai", 0.904),
        (1473504, "Jaipur", 0.722),
        (1473505, "Bengaluru", 0.588),
        (1473506, "Ahmedabad", 0.741),
        (1473507, "Lucknow", 0.172),
        (1473508, "Hyderabad", 0.564),
        (1473509, "Hyderabad", 0.564),
        (1473510, "Kolkata", 0.456),
        (1473511, "Kolkata", 0.456),
    ]

    bias_value = next((entry[2] for entry in data if entry[0] == match_id), None)

    if 0.4 <= bias_value < 0.6:
        num_bowlers = 4
    elif bias_value < 0.4:
        num_bowlers = 5
    elif 0.6 <= bias_value:
        num_bowlers = 3


    
    # Create a copy of the DataFrame to avoid modifying the original
    players_df = players_df.copy()

    # Add 10 points to Fielders
    fielding_players = {
        ("Ravindra Jadeja", "CSK"),
        ("Rohit Sharma", "MI"),
        ("Faf du Plessis", "DC"),
        ("Riyan Parag", "RR"),
        ("Ajinkya Rahane", "KKR"),
        ("Suryakumar Yadav", "MI"),
        ("David Miller", "LSG"),
        ("Ruturaj Gaikwad", "CSK"),
        ("Hardik Pandya", "MI"),
        ("Glenn Maxwell", "PBKS"),
        ("Trent Boult", "MI"),
        ("Andre Russell", "KKR"),
        ("Tilak Varma", "MI"),
        ("Aiden Markram", "LSG"),
        ("Yashasvi Jaiswal", "RR"),
        ("Axar Patel", "DC"),
        ("Nitish Rana", "RR"),
        ("Virat Kohli", "RCB"),
        ("Shreyas Iyer", "PBKS"),
        ("Rinku Singh", "KKR"),
        ("Manish Pandey", "KKR"),
    }

    # Add 20 points to WK
    players_df.loc[players_df["Player Type"] == "WK", "Fantasy_Score"] += 20

    # Boost Fantasy Score for specific players
    players_df.loc[
        players_df[["Player Name", "Team"]].apply(tuple, axis=1).isin(fielding_players), 
        "Fantasy_Score"
    ] += 10

    players = players_df.to_dict('records')
    prob = pulp.LpProblem("Dream_11_IPL_Team_Selection", pulp.LpMaximize)

    num_players = len(players)

    # Decision variables:
    x = pulp.LpVariable.dicts("Select", range(num_players), cat=pulp.LpBinary)

    # Constraints
    prob += pulp.lpSum([x[i] for i in range(num_players)]) == 11, "Total_players"
    prob += pulp.lpSum([players[i]["Credits"] * x[i] for i in range(num_players)]) <= 100, "Credit_limit"

    # At least 1 player from each Player Type category (except BOWL)
    for role in ["WK", "BAT", "ALL"]:
        prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Player Type"] == role]) >= 1, f"Role_{role}_constraint"

    # Exactly num_bowlers must be selected
    prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Player Type"] == "BOWL"]) == num_bowlers, "Role_BOWL_constraint"

    # At least 1 bowler from each team
    teams = set(player["Team"] for player in players)
    for team in teams:
        prob += pulp.lpSum([x[i] for i in range(num_players) if players[i]["Team"] == team and players[i]["Player Type"] == "BOWL"]) >= 1, f"At_least_one_bowler_from_{team}"

    # Objective function: maximize points considering only Fantasy Score
    objective = pulp.lpSum([
        players[i]["Fantasy_Score"] * x[i]  # Only consider Fantasy Score
        for i in range(num_players)
    ])
    prob += objective

    # Solve the problem
    solver = pulp.GLPK_CMD(msg=0)
    prob.solve(solver)

    if pulp.LpStatus[prob.status] == "Optimal":
        selected_players = []
        for i in range(num_players):
            if x[i].varValue == 1:
                selected_players.append({
                    "Player Name": players[i]["Player Name"],
                    "Team": players[i]["Team"],
                    "Type": players[i]["Player Type"],
                    "Score": players[i]["Fantasy_Score"],
                    "Probability": players[i]["probability"],  # Keep probability for sorting
                })

        # Sort players by score, then by probability if scores are the same
        selected_players.sort(key=lambda x: (x["Score"], x["Probability"]), reverse=True)

        # Assign Captain and Vice-Captain
        if len(selected_players) > 0:
            selected_players[0]["Designation"] = "(C)"  # Captain
        if len(selected_players) > 1:
            selected_players[1]["Designation"] = "(VC)"  # Vice-Captain

        return pd.DataFrame(selected_players)
    else:
        return pd.DataFrame()



# import os
# import sys
# sys.path.append("..")

# import pandas as pd
# import numpy as np

# # Load the CSV file into a DataFrame
# df = pd.read_csv('SquadPlayerNames_IndianT20League - SquadData_AllTeams.csv')  # Replace with your actual CSV file name

match_id = 1473439
team = select_dream_team(df, match_id)
print(team)

           Player Name Team  Type  Score  Probability Designation
0      Ramandeep Singh  KKR   ALL     99     0.208703         (C)
1       Ajinkya Rahane  KKR   BAT     97     0.933073        (VC)
2            Moeen Ali  KKR   ALL     94     0.410267         NaN
3        Manish Pandey  KKR   BAT     93     0.411364         NaN
4           Yash Dayal  RCB  BOWL     91     0.063219         NaN
5        Andre Russell  KKR   ALL     86     0.302513         NaN
6      Luvnith Sisodia  KKR    WK     84     0.701214         NaN
7       Venkatesh Iyer  KKR   BAT     83     0.829405         NaN
8     Abhinandan Singh  RCB  BOWL     78     0.821567         NaN
9   Varun Chakravarthy  KKR  BOWL     77     0.189489         NaN
10   Bhuvneshwar Kumar  RCB  BOWL     73     0.293499         NaN


In [ ]:
data = [
        (1473438, "Kolkata", 0.456),
        (1473439, "Hyderabad", 0.564),
        (1473440, "Chennai", 0.355),
        (1473441, "Visakhapatnam", 0.364),
        (1473442, "Ahmedabad", 0.741),
        (1473443, "Guwahati", 0.572),
        (1473444, "Hyderabad", 0.564),
        (1473445, "Chennai", 0.355),
        (1473446, "Ahmedabad", 0.741),
        (1473447, "Visakhapatnam", 0.364),
        (1473448, "Guwahati", 0.572),
        (1473449, "Mumbai", 0.904),
        (1473450, "Lucknow", 0.172),
        (1473451, "Bengaluru", 0.588),
        (1473452, "Kolkata", 0.456),
        (1473453, "Lucknow", 0.172),
        (1473454, "Chennai", 0.355),
        (1473455, "Mohali", 0.593),
        (1473456, "Kolkata", 0.456),
        (1473457, "Hyderabad", 0.564),
        (1473458, "Mumbai", 0.904),
        (1473459, "Mohali", 0.593),
        (1473460, "Ahmedabad", 0.741),
        (1473461, "Bengaluru", 0.588),
        (1473462, "Chennai", 0.355),
        (1473463, "Lucknow", 0.172),
        (1473464, "Hyderabad", 0.564),
        (1473465, "Jaipur", 0.722),
        (1473466, "Delhi", 0.325),
        (1473467, "Lucknow", 0.172),
        (1473468, "Mohali", 0.593),
        (1473469, "Delhi", 0.325),
        (1473470, "Mumbai", 0.904),
        (1473471, "Bengaluru", 0.588),
        (1473472, "Ahmedabad", 0.741),
        (1473473, "Jaipur", 0.722),
        (1473474, "Mohali", 0.593),
        (1473475, "Mumbai", 0.904),
        (1473476, "Kolkata", 0.456),
        (1473477, "Lucknow", 0.172),
        (1473478, "Hyderabad", 0.564),
        (1473479, "Bengaluru", 0.588),
        (1473480, "Chennai", 0.355),
        (1473481, "Kolkata", 0.456),
        (1473482, "Mumbai", 0.904),
        (1473483, "Delhi", 0.325),
        (1473484, "Jaipur", 0.722),
        (1473485, "Delhi", 0.325),
        (1473486, "Chennai", 0.355),
        (1473487, "Jaipur", 0.722),
        (1473488, "Ahmedabad", 0.741),
        (1473489, "Bengaluru", 0.588),
        (1473490, "Kolkata", 0.456),
        (1473491, "Dharamsala", 0.387),
        (1473492, "Hyderabad", 0.564),
        (1473493, "Mumbai", 0.904),
        (1473494, "Kolkata", 0.456),
        (1473495, "Dharamsala", 0.387),
        (1473496, "Lucknow", 0.172),
        (1473497, "Hyderabad", 0.564),
        (1473498, "Dharamsala", 0.387),
        (1473499, "Delhi", 0.325),
        (1473500, "Chennai", 0.355),
        (1473501, "Bengaluru", 0.588),
        (1473502, "Ahmedabad", 0.741),
        (1473503, "Mumbai", 0.904),
        (1473504, "Jaipur", 0.722),
        (1473505, "Bengaluru", 0.588),
        (1473506, "Ahmedabad", 0.741),
        (1473507, "Lucknow", 0.172),
        (1473508, "Hyderabad", 0.564),
        (1473509, "Hyderabad", 0.564),
        (1473510, "Kolkata", 0.456),
        (1473511, "Kolkata", 0.456),
    ]

match_id = 1473493

bias_value = next((entry[2] for entry in data if entry[0] == match_id), None)

if 0.4 <= bias_value < 0.6:
    num_bowlers = 4
elif bias_value < 0.4:
    num_bowlers = 5
elif 0.6 <= bias_value < 0.7:
    num_bowlers = 3
else:
    num_bowlers = 2  # Optional: handle cases where bias_value is 0.7 or higher

print(num_bowlers)

In [ ]:
def calculate_num_batsmen(batting_pitch_bias):
    # Calculate the number of batsmen based on the bias
    if batting_pitch_bias < 0.5:
        num_bowlers = 5  # For bias between 0.3 and 0.5
    elif 0.5 <= batting_pitch_bias < 0.8:
        num_bowlers = 4 if batting_pitch_bias == 0.5 else 4  # For bias between 0.5 and 0.8
    else:
        num_bowlers = 3  # For bias greater than 0.8

    return num_bowlers

In [ ]:
num_bowlers

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('ground_data_bias.csv')  # Replace with your actual CSV file name

# Sort the DataFrame by the 'Bias' column
sorted_df = df.sort_values(by='Bias')  # You can set ascending=False to sort in descending order

# Display the sorted DataFrame (optional)
print("Sorted DataFrame:")
print(sorted_df)

# Save the sorted DataFrame to a new CSV file
sorted_df.to_csv('ground_data_bias.csv', index=False)  # Replace with your desired output file name

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('ground_data_bias.csv')  # Replace with your actual CSV file name

# Check if the 'Bias' column exists
if 'Bias' in df.columns:
    # Subtract 0.1 from Bias values greater than 0.5
    df.loc[df['Bias'] > 0.5, 'Bias'] += 0.2

    # Display the updated DataFrame (optional)
    print("Updated DataFrame:")
    print(df)

    # Save the updated DataFrame back to the CSV file
    df.to_csv('ground_data_bias.csv', index=False)  # Replace with your desired output file name
else:
    print("The 'Bias' column is not present in the CSV file.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
df = pd.read_csv('ground_data_bias.csv')  # Replace with your actual CSV file name

# Check if the 'Bias' column exists
if 'Bias' in df.columns:
    # Calculate the median of the Bias column
    median_bias = df['Bias'].median()
    print(f"The median of the Bias column is: {median_bias:.2f}")

    # Plot the distribution of the Bias column
    plt.figure(figsize=(10, 6))
    plt.hist(df['Bias'], bins=10, color='skyblue', edgecolor='black', alpha=0.7)
    plt.axvline(median_bias, color='red', linestyle='dashed', linewidth=1, label=f'Median: {median_bias:.2f}')
    plt.title('Distribution of Bias')
    plt.xlabel('Bias')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(axis='y', alpha=0.75)
    plt.tight_layout()  # Adjust layout to prevent clipping
    plt.show()
else:
    print("The 'Bias' column is not present in the CSV file.")

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('ipl_2025_schedule.csv')  # Replace with your actual CSV file name

# Check if the 'Venue' column exists
if 'Venue' in df.columns:
    # Get unique entries in the 'Venue' column
    unique_venues = df['Venue'].unique()

    # Display the unique venues
    print("Unique venues:")
    for venue in unique_venues:
        print(venue)
else:
    print("The 'Venue' column is not present in the CSV file.")